In [1]:
import graphviz as gv
import numpy as np
import pandas as pd
import heapq as hq
import math

In [2]:
def SetList(list,name):
  fh = open(name)
  line1 = True

  for line in fh:
    if line1:
      line1=False
      continue
    data = line.rstrip().split(',')
    toAdd = (int(data[0]),int(data[1]))
    list.append(toAdd)
  return list

In [3]:
pointsAlmacenes = list()
pointsEntrega = list()
extraPoints = list()
pointsAlmacenes = SetList(pointsAlmacenes, "pointsAlmacenes.csv")
pointsEntrega = SetList(pointsEntrega, "pointsEntrega.csv")
extraPoints = SetList(extraPoints, "extraPoints.csv")

In [4]:
def DeleteEqualsInExtraPoints():
  i=0
  j=0
  k=0
  nodes = list()
  almacen = False
  while i < len(extraPoints):
    addSomething = False
    toAdd=list()
    number = extraPoints[i][0]+extraPoints[i][1]*nFilas
    #Nodo de Abajo
    if number-nFilas>=0:
      toAdd.append(number-nFilas)

    #Nodo de la Izquierda
    if number-1>= (extraPoints[i][1]*nFilas):
      toAdd.append(number-1)

    #Nodo de la Derecha
    if number+1<((extraPoints[i][1]+1)*nFilas):
      toAdd.append(number+1)

    #Nodo de Arriba
    if number+nFilas<(nFilas*nColumnas):
      toAdd.append(number+nFilas)

    while j < len(pointsEntrega):
      if extraPoints[i][0]==pointsEntrega[j][0] and extraPoints[i][1]==pointsEntrega[j][1]:
        extraPoints.pop(i)
        i-=1
        j+=1
        toAdd.append(1)
        addSomething = True
        entrega.append(number)   
        break
      else:
        break
    while k < len(pointsAlmacenes):
      if extraPoints[i][0]==pointsAlmacenes[k][0] and extraPoints[i][1]==pointsAlmacenes[k][1]:
        extraPoints.pop(i)
        i-=1
        k+=1
        toAdd.append(2)
        addSomething = True
        almacenes.append(number)
        break
      else:
        break
    if addSomething == False:
      toAdd.append(0)
    i+=1
    toAdd.append(number)
    nodes.append(toAdd)
  return nodes

In [5]:
nFilas = nColumnas = 1000
almacenes = list()
entrega= list()
nodos = DeleteEqualsInExtraPoints()

In [6]:
def DBP(a,b):
  x1, y1=a
  x2, y2=b
  return abs(x1-x2)+ abs(y1-y2)

In [7]:
group = [-1]*len(pointsEntrega)
group2 = [-1]*len(extraPoints)

for i in range(len(pointsEntrega)):
  m=0
  d = DBP(pointsEntrega[i],pointsAlmacenes[0])
  for j in range(len(pointsAlmacenes)):
    d2 = DBP(pointsEntrega[i],pointsAlmacenes[j])
    if d2 <d:
      d = d2
      m = j
  group[i]=m

for i in range(len(extraPoints)):
  m=0
  d = DBP(extraPoints[i],pointsAlmacenes[0])
  for j in range(len(pointsAlmacenes)):
    d2 = DBP(extraPoints[i],pointsAlmacenes[j])
    if d2 <d:
      d = d2
      m = j
  group2[i]=m

In [8]:
nodesGroups = list()
for i in range(len(pointsAlmacenes)):
  toAdd=list()
  almacen = pointsAlmacenes[i][0]+pointsAlmacenes[i][1]*nFilas
  toAdd.append(nodos[almacen])
  for j in range(len(group)):
    if group[j]==i:
      entrega = pointsEntrega[j][0]+pointsEntrega[j][1]*nFilas
      toAdd.append(nodos[entrega])
  for j in range(len(group2)):
    if group2[j]==i:
      entrega = extraPoints[j][0]+extraPoints[j][1]*nFilas
      toAdd.append(nodos[entrega])
  nodesGroups.append(toAdd)

In [9]:
for i in range(len(nodesGroups)):
  j=i
  while j<len(nodesGroups)-1:
    j+=1
    if len(nodesGroups[i])>len(nodesGroups[j]):
      nodesGroups[i],nodesGroups[j]=nodesGroups[j],nodesGroups[i]

NodosLucia = list()
NodosJorge = list()
NodosLuis = list()
NodosDaniel = list()
NodosJoaquin = list()

for i in range(len(nodesGroups)):
  if i<18:
    NodosLucia.append(nodesGroups[i])
  elif i<36:
    NodosJorge.append(nodesGroups[i])
  elif i<55:
    NodosLuis.append(nodesGroups[i])
  elif i<74:
    NodosDaniel.append(nodesGroups[i])
  else:
    NodosJoaquin.append(nodesGroups[i])

In [10]:
def convertWithWeight(n):
    newList = []
    for i in range(len(n)):
        newList.append([])
        newList[i].append(n[i])
        newList[i].append(1)
    newList.pop(-1)
    newList.pop(-1)
    return newList

In [11]:
nodosJoaquinConPeso = []
for i in range(len(NodosJoaquin)):
    nodosJoaquinConPeso.append([])
    for j in range(len(NodosJoaquin[i])):
        nodosJoaquinConPeso[i].append(convertWithWeight(NodosJoaquin[i][j]))

In [12]:
nodosConPeso = []
for i in range(len(nodos)):
    nodosConPeso.append(convertWithWeight(nodos[i]))

In [14]:
def dijkstra(G, s):
  
  n = len(G)
  visited = [False]*n
  path = [None]*n
  cost = [math.inf]*n
  cost[s] = 0
  queue = [(0, s)]
  
  while queue:
    g_u, u = hq.heappop(queue)
    if not visited[u]:
      visited[u] = True
      for v, w in G[u]:
        f = g_u + w
        if f < cost[v]:
          cost[v] = f
          path[v] = u
          hq.heappush(queue, (f, v))
          
  return path, cost

In [26]:
#Aplicamos el algoritmo a todos los nodos de un grupo
almacen = 0
for i in range(len(NodosJoaquin)):
    for j in range(len(NodosJoaquin[i])):
        if NodosJoaquin[i][j][-2] == 2:
            almacen = NodosJoaquin[i][j][-1]
            print(almacen)
            dijkstra(nodosConPeso, almacen)

750646
242187
881821
685537
408369
986396
155876
527595
417518
934079
818141
232101
86331
640393
813452
665070
9753
174415
206623
